# Postprocessing

After training the model and using it to predict submission cases (210-300), in order to send a submission, we must be able to "inverse" the processing in order to generate segmentation nifti file that matches voxel dimension and size of the original image, to do that we must store voxdim and shape of the unprocessed test cases.

Let's create a json to keep track of essential data in order to complete the submission

In [ ]:
import pickle
import numpy as np
import skimage.transform
from scipy.ndimage import zoom
import matplotlib.pyplot as plt

In [ ]:
data_path = "data/case_"
imaging_fname = "/imaging.nii.gz"
segmentation_fname = "/segmentation.nii.gz"

class Patient:
  def __init__(self, number):
      self.number = number
      
      self.imaging_path = self.get_im_data_path()
      self.imaging = nib.load(self.imaging_path)
      
      if number < 210: # segmentation file only exists for non-submission cases
        self.segmentation_path = self.get_seg_data_path()
        self.segmentation = nib.load(self.segmentation_path)

  def get_im_data_path(self):
      '''
      returns path to imaging file
      '''
      out = "00000"

      out = out[len(str(self.number)):]
      out += str(self.number)
      return data_path + out + imaging_fname
  
  def get_seg_data_path(self):
      '''
      returns path to segmentation file
      '''
      out = "00000"

      out = out[len(str(self.number)):]
      out += str(self.number)
      return data_path + out + segmentation_fname

  def rr(self, target_file, target_resolution=[1,1,1], target_shape=[192,192,192]):
    '''
    Target file can be either self.imaging or self.segmentation
    Function that returns reshaped and resized target block
    ''' 
    target_fig = target_file.get_fdata()

    # x,y,z
    vox_dim = (target_file.header['pixdim'][2], target_file.header['pixdim'][3], target_file.header['pixdim'][1])

    scale_vector = (vox_dim[0]/target_resolution[0],
                    vox_dim[1]/target_resolution[1],
                    vox_dim[2]/target_resolution[2])

    

    iso_target_fig = skimage.transform.rescale(target_fig, scale_vector, order=3, preserve_range=True,  mode='constant')
    
    # print(iso_target_fig.dtype) # float64
    
    factors = ( target_shape[0]/iso_target_fig.shape[0],
                target_shape[1]/iso_target_fig.shape[1],
                target_shape[2]/iso_target_fig.shape[2],)

    rr_target_cube = zoom(iso_target_fig, factors, order=3, mode='nearest')

    # print(rr_target_cube.dtype) # float64

    # rr = rescaled and reshaped
    # rr_target = nib.Nifti1Image(rr_target_fig, target_file.affine)

    # # setto le dimensioni manualmente, sono abbastanza sicuro che il rescaling sia avvenuto correttamente

    # for i in range(3):
    #   rr_target.header['pixdim'][i+1] = target_resolution[i]

    # for i in range(3):
    #   rr_target.affine[i][2-i] = -target_resolution[i]

    return rr_target_cube

  def process_imaging(self, target_resolution=[1,1,1], target_shape=[192,256,256]):
    '''
    function that returns reshaped, resized, normalized imaging block
    '''

    rr_imaging = self.rr(self.imaging, target_resolution, target_shape)
    output = cv2.normalize(rr_imaging, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_8UC1)
    return output

  def process_segmentation(self, target_resolution=[1,1,1], target_shape=[192,256,256]):
    '''
    function that returns reshaped, resized segmentation block
    '''

    output = self.rr(self.segmentation, target_resolution, target_shape)

    # might break things (it does, kinda)
    #output = cv2.normalize(rr_segmentation, None, alpha = 0, beta = 3, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_8UC1)
    
    output = np.rint(output)
    output = output.astype('uint8')

    return output

def get_number(n):
  '''
  returns num padded with zeros
  '''
  out = "00000"

  out = out[len(str(n)):]
  out += str(n)
  return out

In [ ]:
record = []

for n_pat in range(0,300):
    pat = Patient(n_pat)

    #print(pat.imaging.header)
    # x, y, z
    shape = [pat.imaging.header['dim'][2], pat.imaging.header['dim'][3], pat.imaging.header['dim'][1]]
    vox_dim = [pat.imaging.header['pixdim'][2], pat.imaging.header['pixdim'][3], pat.imaging.header['pixdim'][1]]
    
    data = {
        'case_id': n_pat,
        'vox_dim': vox_dim,
        'shape': shape,
    }

    record.append(data)

with open('original_dims.pkl', 'wb') as f:
    pickle.dump(record, f)

### Provo con 1 caso (ID168)

In [ ]:
with open('original_dims.pkl', 'rb') as f:
    original_dims = pickle.load(f)

patVolList = []

path = 'D:\\kits19\\processed_data\\predict_aug_09_12\\'

for filename in os.listdir(path):
    print(filename)
    img = Image.open(str(path+filename))
    slice = np.asarray(img)
    patVolList.append(slice)

pat168vol = np.array(patVolList)


Function that revert effects of preprocessing

In [ ]:
def revert(volume, n_pat):
    
    vox_dim = original_dims[n_pat]['vox_dim']
    shape = original_dims[n_pat]['shape']

    # print(vox_dim)
    # z x y
    scale_vector = (1/vox_dim[2],
                    1/vox_dim[0],
                    1/vox_dim[1])
    # print(scale_vector)

    iso_target_fig = skimage.transform.rescale(volume, scale_vector, order=3, preserve_range=True,  mode='constant')

    # z x y
    factors = ( shape[2]/iso_target_fig.shape[0],
                shape[0]/iso_target_fig.shape[1],
                shape[1]/iso_target_fig.shape[2],)

    output = zoom(iso_target_fig, factors, order=3, mode='nearest')
    

    return output

In [ ]:
original_form = revert(pat168vol, 168)

Evaluate single case

In [ ]:
from starter_code import evaluation

print(evaluation.evaluate(168, original_form))

# Evaluate all test cases

In [ ]:
with open('original_dims.pkl', 'rb') as f:
    original_dims = pickle.load(f)


path = 'D:\\kits19\\predicts\\batch_predict_tumor_11_12\\'

listOfVolumes = []

for num_pat in range(168, 210):
    volume = []
    case_file = 'case_' + get_number(num_pat) + '_predict_img\\'
    
    for filename in os.listdir(str(path+case_file)):
        img = Image.open(str(path+case_file+filename))
        npimg = np.asarray(img)
        volume.append(npimg)
    listOfVolumes.append(volume)

volumesToRevert = []  # list of ndarray
for volume in listOfVolumes:
    volumesToRevert.append(np.array(volume))

Revert all prediction ndarrays into original shape and vox_dim

In [ ]:
def revertGenerator():
    for i in range(168,210):
        readyVol = revert(volumesToRevert[i-168], i)
        yield (i, readyVol)

#generator object
evalReadyVolGen = revertGenerator()

Finally evaluate all test_imgs

In [ ]:
from starter_code import evaluation

kidneyScoreList = []

for i in range(168,210):
    predict_data = next(evalReadyVolGen)
    print(predict_data[0])
    kidneyScoreList.append(evaluation.evaluate(predict_data[0], predict_data[1])[1])
    print(kidneyScoreList[-1])

partial_sum = 0
for val in kidneyScoreList:
    partial_sum += val

print('Average score:', partial_sum/len(kidneyScoreList))

# 57% kidney
# 21% cancer

---
Create folder for evaluation

In [ ]:
import pickle

with open('original_dims.pkl', 'rb') as f:
    original_dims = pickle.load(f)


path = 'D:\\kits19\\predicts\\sub_img_kidney_1\\'


for num_pat in range(210, 300):
    volume = []
    case_file = 'case_' + get_number(num_pat) + '_predict_img\\'
    
    for filename in os.listdir(str(path+case_file)):
        img = Image.open(str(path+case_file+filename))
        npimg = np.asarray(img)
        volume.append(npimg)
    
    volumeToRevert = np.array(volume)
    evalReadyVolume = revert(volumeToRevert, num_pat)

    evalReadyVolume[evalReadyVolume<0.7]=0
    evalReadyVolume[evalReadyVolume>=0.7]=1

    pat = Patient(num_pat)
    final_predict = nib.Nifti1Image(evalReadyVolume, affine=pat.imaging.affine)

    nib.save(final_predict, os.path.join("D:\\kits19\\submission_ready\\predictions", str("prediction_ "+get_number(num_pat)+".nii.gz")))

---

# Plot results
- first for kidney model
- then for tumor model

In [ ]:
predict_168_65 = np.asanyarray(Image.open('D:\\kits19\\predicts\\batch_predict_09_12_56\\case_00168_predict_img\\case_00168_kid_00065_predict.png'))
actual_168_65 = np.asanyarray(Image.open('D:\\kits19\\processed_data\\test_kidney_labels\\case_00168_seg_00065.png'))
imaging_168_65 = np.asanyarray(Image.open('D:\\kits19\\processed_data\\test_images\\case_00168_test_img\\case_00168_im_00065.png'))


In [ ]:

fig = plt.figure(figsize=(8,8))
a = fig.add_subplot(1,3,1)
imgplot = plt.imshow(imaging_168_65[:,:], cmap = 'gray')
plt.title("Image")

a = fig.add_subplot(1,3,2)
imgplot = plt.imshow(predict_168_65[:,:], cmap='gray')
plt.title("Predicted Segmentation")

a = fig.add_subplot(1,3,3)
imgplot = plt.imshow(actual_168_65[:,:], cmap='gray')
plt.title("Actual Segmentation")

---

In [ ]:
predict_197_61 = np.asanyarray(Image.open('D:\\kits19\\predicts\\batch_predict_tumor_11_12\\case_00197_predict_img\\case_00197_kid_00061_predict.png'))
actual_197_61 = np.asanyarray(Image.open('D:\\kits19\\processed_data\\test_tumor_labels\\case_00197_tum_00061.png'))
imaging_197_61 = np.asanyarray(Image.open('D:\\kits19\\processed_data\\test_images\\case_00197_test_img\\case_00197_im_00061.png'))

In [ ]:
fig = plt.figure(figsize=(8,8))
a = fig.add_subplot(1,3,1)
imgplot = plt.imshow(imaging_197_61[:,:], cmap = 'gray')
plt.title("Image")

a = fig.add_subplot(1,3,2)
imgplot = plt.imshow(predict_197_61[:,:], cmap='gray')
plt.title("Predicted Segmentation")

a = fig.add_subplot(1,3,3)
imgplot = plt.imshow(actual_197_61[:,:], cmap='gray')
plt.title("Actual Segmentation")

In [ ]:
predict_196_70 = np.asanyarray(Image.open('D:\\kits19\\predicts\\batch_predict_tumor_11_12\\case_00196_predict_img\\case_00196_kid_00070_predict.png'))
actual_196_70 = np.asanyarray(Image.open('D:\\kits19\\processed_data\\test_tumor_labels\\case_00196_tum_00070.png'))
imaging_196_70 = np.asanyarray(Image.open('D:\\kits19\\processed_data\\test_images\\case_00196_test_img\\case_00196_im_00070.png'))

In [ ]:
fig = plt.figure(figsize=(8,8))
a = fig.add_subplot(1,3,1)
imgplot = plt.imshow(imaging_196_70[:,:], cmap = 'gray')
plt.title("Image")

a = fig.add_subplot(1,3,2)
imgplot = plt.imshow(predict_196_70[:,:], cmap='gray')
plt.title("Predicted Segmentation")

a = fig.add_subplot(1,3,3)
imgplot = plt.imshow(actual_196_70[:,:], cmap='gray')
plt.title("Actual Segmentation")